## importing the libraries

In [1]:
import pandas as pd
import numpy as np
import requests

## read the JSON file saved in ex02

In [2]:
df = pd.read_json('../data/auto.json', orient='records')

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CarNumber  725 non-null    object 
 1   Refund     725 non-null    int64  
 2   Fines      725 non-null    float64
 3   Make       725 non-null    object 
 4   Model      716 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 28.4+ KB


## enrich the dataframe using a sample from that dataframe

In [5]:
sample_df = df.sample(n=200, random_state=21).reset_index(drop=True)
sample_df.head()

,CarNumber,Refund,Fines,Make,Model
0,M0299X197RUS,2,19200.00,Ford,Focus
1,83298C154RUS,2,8594.59,Ford,Focus
2,H957HY161RUS,1,2000.00,Ford,Focus
3,T941CC96RUS,1,2000.00,Ford,Focus
4,H966HY161RUS,1,500.00,Ford,Focus


In [6]:
concat_rows = pd.concat([df, sample_df])
concat_rows.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


In [7]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
dtype: int64

## enrich the dataframe concat_rows by a new column with the data generated

In [8]:
np.random.seed(21)
Year = pd.Series([np.random.randint(1980, 2020) for i in range(len(concat_rows))], name='Year')
fines = concat_rows
fines['Year'] = Year
fines.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014


## enrich the dataframe with the data from another dataframe

In [9]:
owners = pd.read_json('../data/surname.json')



In [10]:
owners.head()

,0,1,2
0,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15


In [11]:
owners.columns = owners.iloc[0]
owners = owners.drop(0)
owners.head()

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72


In [12]:
owners.columns

Index(['NAME', 'COUNT', 'RANK'], dtype='object', name=0)

In [13]:
owners = owners[owners['NAME'].str.contains('[a-zA-Z]')]
surname = [np.random.choice(owners['NAME']) for x in range(len(fines['CarNumber'].drop_duplicates()))]
owners = pd.DataFrame(fines['CarNumber'].drop_duplicates().copy())
owners['SURNAME'] = surname
owners

,CarNumber,SURNAME
0,Y163O8161RUS,BAKER
1,E432XX77RUS,CRUZ
2,7184TT36RUS,MARTIN
3,X582HE161RUS,REED
4,92918M178RUS,COOPER
...,...,...
715,O136HO197RUS,HOWARD
719,O22097197RUS,EVANS
721,M0309X197RUS,ROGERS
722,O673E8197RUS,WILSON


In [14]:
def gen_num(n):
    letters = 'ABCEXKH'
    end = 'RUS'
    res = list()
    for _ in range(n):
        rand = np.random.randint(0, 1000, 2)
        rand_letters = np.random.choice(list(letters), 3)
        num = f'{rand_letters[0]}{rand[0]:03}{rand_letters[1]}{rand_letters[2]}{rand[1]:03}{end}'
        res.append(num)
    return res

In [15]:
fine_df = pd.DataFrame(
    [fines[['Make', 'Model']].iloc[np.random.randint(0, len(fines))] for i in range(5)])
fine_df['CarNumber'] = gen_num(5)
fine_df['Refund'] = [np.random.choice(fines['Refund']) for i in range(5)]
fine_df['Fines'] = [np.random.choice(fines['Fines']) for i in range(5)]
fine_df['Year'] = [np.random.choice(fines['Year']) for i in range(5)]
fine_df = fine_df[['CarNumber', 'Refund','Fines', 'Make', 'Model', 'Year']]
fine_df

,CarNumber,Refund,Fines,Make,Model,Year
397,H799AX131RUS,2,600.00,Toyota,Camry,1991
176,B090KH243RUS,1,500.00,Ford,Focus,2018
386,C280EC929RUS,2,1500.00,Ford,Focus,2011
720,K202AE490RUS,2,2000.00,Ford,Focus,2012
325,K008XE145RUS,2,3400.00,Ford,Focus,2010


In [16]:
fines = pd.concat([fines, fine_df], ignore_index=True)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
925,H799AX131RUS,2,600.00,Toyota,Camry,1991
926,B090KH243RUS,1,500.00,Ford,Focus,2018
927,C280EC929RUS,2,1500.00,Ford,Focus,2011
928,K202AE490RUS,2,2000.00,Ford,Focus,2012


In [17]:
owners = owners[:-20]
owners

,CarNumber,SURNAME
0,Y163O8161RUS,BAKER
1,E432XX77RUS,CRUZ
2,7184TT36RUS,MARTIN
3,X582HE161RUS,REED
4,92918M178RUS,COOPER
...,...,...
681,T914CT197RUS,BAILEY
682,E41977152RUS,EDWARDS
684,9464EX178RUS,RIVERA
685,O50197197RUS,JACKSON


In [18]:
new_data = pd.DataFrame({'CarNumber': gen_num(3),'SURNAME': [np.random.choice(owners['SURNAME']) for i in range(3)]})
owners = pd.concat([owners, new_data])
owners

,CarNumber,SURNAME
0,Y163O8161RUS,BAKER
1,E432XX77RUS,CRUZ
2,7184TT36RUS,MARTIN
3,X582HE161RUS,REED
4,92918M178RUS,COOPER
...,...,...
685,O50197197RUS,JACKSON
686,7608EE777RUS,MILLER
0,K610KH248RUS,ROBINSON
1,K776CH330RUS,WARD


In [19]:
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
925,H799AX131RUS,2,600.00,Toyota,Camry,1991
926,B090KH243RUS,1,500.00,Ford,Focus,2018
927,C280EC929RUS,2,1500.00,Ford,Focus,2011
928,K202AE490RUS,2,2000.00,Ford,Focus,2012


In [20]:
# join both dataframes
# the new dataframe should have only the car numbers that exist in both dataframes
res1 = fines.merge(owners, how='inner')
res1

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,BAKER
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,BAKER
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,CRUZ
3,E432XX77RUS,2,13000.00,Toyota,Camry,2018,CRUZ
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MARTIN
...,...,...,...,...,...,...,...
894,E41977152RUS,2,2400.00,Ford,Focus,1989,EDWARDS
895,9464EX178RUS,2,2100.00,Ford,Focus,1988,RIVERA
896,O50197197RUS,2,7800.00,Ford,Focus,1992,JACKSON
897,7608EE777RUS,1,4000.00,Skoda,Octavia,2000,MILLER


In [21]:
# the new dataframe should have all the car numbers that exist in both dataframes
res2 = fines.merge(owners, how='outer')
res2

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,BAKER
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1980.00,BAKER
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,CRUZ
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,2018.00,CRUZ
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MARTIN
...,...,...,...,...,...,...,...
928,K202AE490RUS,2.00,2000.00,Ford,Focus,2012.00,NaN
929,K008XE145RUS,2.00,3400.00,Ford,Focus,2010.00,NaN
930,K610KH248RUS,NaN,NaN,NaN,NaN,NaN,ROBINSON
931,K776CH330RUS,NaN,NaN,NaN,NaN,NaN,WARD


In [22]:
# the new dataframe should have only the car numbers from the fines dataframe
res3 = fines.merge(owners, how='left')

In [23]:
# the new dataframe should have only the car numbers from the owners dataframe
res4 = fines.merge(owners, how='right')
res4

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,BAKER
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1980.00,BAKER
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,CRUZ
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,2018.00,CRUZ
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MARTIN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2000.00,MILLER
898,7608EE777RUS,1.00,4000.00,Skoda,Octavia,1996.00,MILLER
899,K610KH248RUS,NaN,NaN,NaN,NaN,NaN,ROBINSON
900,K776CH330RUS,NaN,NaN,NaN,NaN,NaN,WARD


## create a pivot table from the fines dataframe

In [24]:
pivot_table = pd.pivot_table(res1,values='Fines',index=['Make', 'Model'],columns=['Year'],aggfunc=np.sum)
pivot_table

Year                   1980      1981      1982     1983     1984      1985  \
Make       Model                                                              
Ford       Focus   74094.59 423283.76 161883.76 97194.59 96589.17 146972.93   
           Mondeo       NaN       NaN       NaN      NaN      NaN       NaN   
Skoda      Octavia  1900.00   8594.59   6900.00 21189.17      NaN  10294.59   
Toyota     Camry   12000.00   8594.59       NaN  7200.00      NaN       NaN   
           Corolla      NaN       NaN   2000.00      NaN      NaN       NaN   
Volkswagen Golf    31900.00       NaN       NaN  8594.59   300.00  24300.00   
           Jetta        NaN   4000.00       NaN      NaN      NaN       NaN   
           Passat       NaN  11100.00       NaN  3200.00 25000.00   5000.00   
           Touareg      NaN       NaN       NaN      NaN      NaN   5800.00   

Year                   1986     1987     1988     1989  ...      2010  \
Make       Model                                        ...             
Ford       Focus   90394.59 80600.00 84194.59 73300.00  ... 125778.35   
           Mondeo       NaN      NaN      NaN  8600.00  ...       NaN   
Skoda      Octavia   600.00  5200.00      NaN 91400.00  ...   3600.00   
Toyota     Camry   22400.00      NaN      NaN 22400.00  ...       NaN   
           Corolla      NaN  8000.00      NaN 11600.00  ...  24000.00   
Volkswagen Golf         NaN  9300.00      NaN  5800.00  ...       NaN   
           Jetta        NaN      NaN      NaN      NaN  ...       NaN   
           Passat  15000.00 12300.00      NaN      NaN  ...   5700.00   
           Touareg      NaN      NaN      NaN      NaN  ...   6300.00   

Year                   2011      2012      2013      2014      2015     2016  \
Make       Model                                                               
Ford       Focus   90789.17 127789.17 148289.17 117394.59 221000.00 96989.17   
           Mondeo       NaN  34400.00       NaN       NaN       NaN 46200.00   
Skoda      Octavia   500.00    500.00  15594.59    300.00  46394.59   300.00   
Toyota     Camry        NaN   8594.59       NaN       NaN       NaN      NaN   
           Corolla  8594.59  30300.00       NaN       NaN       NaN  3400.00   
Volkswagen Golf      300.00       NaN       NaN       NaN   2300.00      NaN   
           Jetta        NaN       NaN       NaN       NaN       NaN      NaN   
           Passat       NaN       NaN       NaN   1600.00   1600.00  2100.00   
           Touareg      NaN       NaN       NaN   1300.00    500.00      NaN   

Year                    2017      2018     2019  
Make       Model                                 
Ford       Focus   233594.59 257994.59 84400.00  
           Mondeo        NaN       NaN      NaN  
Skoda      Octavia       NaN 156200.00  9500.00  
Toyota     Camry         NaN  14000.00 18100.00  
           Corolla   9600.00       NaN      NaN  
Volkswagen Golf          NaN       NaN   300.00  
           Jetta         NaN       NaN      NaN  
           Passat        NaN       NaN  9900.00  
           Touareg       NaN       NaN      NaN  

[9 rows x 40 columns]

## • save both the fines and owners dataframes to CSV files without an index

In [25]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)